In [2]:
import numpy as np
import math

In [21]:
h_0 = 1.4*(10**(-22))
h_m1 = 2.3*(10**(-26))
h_m2 = 3.3*(10**(-31))

dt = 60

pi = math.pi

c = 299792458

e1 = (h_0/2)*dt + 2*h_m1*(dt*dt) + (2/3)*((pi*h_m2*dt)**2)*dt
e2 = h_m1*dt + h_m2*(pi*dt)**2
e3 = (h_0/(2*dt))+ 4*h_m1 + (8/3)*(pi*pi)*h_m2*dt

Q = np.array([[e1,e2],[e2,e3]])
print(Q)

[[4.36560000e-21 1.39172509e-24]
 [1.39172509e-24 1.25918778e-24]]


In [4]:
def time_update(x, P):
  # define Q
  h_0 = 1.4*(10**(-22))
  h_m1 = 2.3*(10**(-26))
  h_m2 = 3.3*(10**(-31))

  dt = 60

  e1 = (h_0/2)*dt + 2*h_m1*(dt*dt) + (2/3)*((pi*h_m2*dt)**2)*dt
  e2 = h_m1*dt + h_m2*(pi*dt)**2
  e3 = (h_0/(2*dt))+ 4*h_m1 + (8/3)*(pi*pi)*h_m2*dt

  Q = np.array([[e1,e2],[e2,e3]])

  # time update steps
  A = np.array([[1, delta_t], [0, 1]])
  x = A @ x
  P = A @ P @ A.T + Q

  return x, P

In [31]:
import time
#phi - angle from x-y plane (phi = pi/2 is z-axis)
#theta - angle on x-y plane from positive y-axis)
def sphere_to_cart(r,phi,theta):
        return r*math.sin(theta)*math.cos(phi),r*math.cos(theta)*math.cos(phi),r*math.sin(phi)

dist = lambda p: (p[0]**2+p[1]**2+p[2]**2)**(0.5)


#Starting with a Moon Frame of Reference, this is a station on the south pole
ss = (1470000,3*pi/2,0)
#For 250km away we have arc length theta*r = dist so we can change angle by dist/r = 250/1740
#We create 5 stations in a cross shape with one in the center on the south pole
stations = [ss,(ss[0],ss[1]+(250/1470),0),(ss[0],ss[1]-(250/1470),0),(ss[0],ss[1]+(250/1470),pi/2),(ss[0],ss[1]-(250/1470),pi/2)]
cart_stations = []
for s in stations:
    x = list(sphere_to_cart(*s))
    #We gave the coords with the moon as frame of reference so we have to shift to keep an earth frame of reference
    x[0]+=6378000+384400000+1740000
    cart_stations.append(np.array(x))

def get_satellite(t):
    #Satellites on the backside of the earth
    theta = np.random.uniform(-3*pi/4,-pi/4)

    #They have to see around the earth so they are far enough from directly behind it
    phi = np.random.choice([-1,1])*np.random.uniform(pi/6,pi/4)

    #Atmosphere range where satellites fly
    r = np.random.uniform(20000000,21000000)

    x,y,z = sphere_to_cart(r,phi,theta)

    #Sat Velo
    v_st = 1944
    A = np.array([[x,y,x*z],[y,-1*y,y*z],[z,0,-1*(x**2+y**2)]])

    theta = np.random.uniform(0,2*pi)
    sat_velo_vector = v_st*(A@(np.array([sphere_to_cart(1,0,theta)]).T))

    return {"time":t,"pos":np.array([x,y,z]),"velo":sat_velo_vector}

def measurement_noise(R,N_i):
     return np.linalg.cholesky(R(N_i))@np.random.normal(np.array([[0 for _ in range(2*N_i)]]).T)

def GPS_measurements(station,R,N_i = 2,t=0,t_offset=0):
    #Random Satellite Coords

    satellites = [get_satellite(t) for _ in range(N_i)]
    
    true_ranges = []
    est_ranges = []


    true_rates = []
    est_rates = []


    #station = (position, Local Time, Local drift)

    stn = station[0]
    

    for sat in satellites:

        #Line of Sight vector between sat and station
        los_vector = stn-sat["pos"]

        true_range = dist(los_vector)

        t_off = true_range/c  
        t_offset += t_off

        true_ranges.append([true_range])

        est_ranges.append([c*(station[1]+t_off - sat['time'])])


        v_sn = 1023

        B = np.array([[stn[0],stn[1],stn[0]*stn[2]],[stn[1],-1*stn[0],stn[1]*stn[2]],[stn[2],0,-1*(stn[0]**2+stn[1]**2)]])
        station_velo_vector = v_sn*(B@(np.array([[0,-1,0]]).T))


        velo_diff = station_velo_vector - sat["velo"]

        psudeorange_rate = (np.dot(velo_diff.T,los_vector)/np.dot(los_vector,los_vector))*los_vector

        true_rates.append([dist(psudeorange_rate)])

        #THIS IS FILLER FIX LATER
        est_rates.append([dist(psudeorange_rate)])
    

    measurments = np.vstack((true_ranges,true_rates))+measurement_noise(R,N_i) - np.vstack((est_ranges,est_rates))
    
    return measurments


t_offset = 0
for station in cart_stations:
    R = lambda n:0.01*np.eye(2*n,2*n)
    t = time.time()
    stn = [station, t+t_offset]
    print(GPS_measurements(stn,R,t=t+t_offset,t_offset = t_offset))

[[-30.31915039]
 [-33.35542232]
 [  0.10282516]
 [ -0.0923996 ]]
[[11.30084431]
 [ 1.29718137]
 [-0.12478638]
 [ 0.06924057]]
[[-8.86475641e+00]
 [ 3.25401136e+01]
 [ 8.95462036e-02]
 [ 1.42765045e-02]]
[[-12.6498667 ]
 [-30.30777282]
 [  0.0378418 ]
 [  0.03406334]]
[[-15.00180411]
 [-32.00340986]
 [ -0.14319611]
 [ -0.09751892]]
